In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

### Originals image path

In [2]:
path = Path('database/originals')

### HSV, YCrCb and LAB paths 

In [3]:
HSV_path = 'tests/HSV'
Path(HSV_path).mkdir(parents=True, exist_ok=True)

YCrCb_path = 'tests/YCrCb'
Path(YCrCb_path).mkdir(parents=True, exist_ok=True)

LAB_path = 'tests/LAB'
Path(LAB_path).mkdir(parents=True, exist_ok=True)

RGB_path = 'tests/RGB'
Path(RGB_path).mkdir(parents=True, exist_ok=True)

### Convert BGR images to HSV/YCrCb/LAB 

In [4]:
def other_colors(file, HSV_path, YCrCb_path, LAB_path, RGB_path): 
    image = cv2.imread(file.__str__())
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    cv2.imwrite(HSV_path + '/' + file.name, hsv)
    cv2.imwrite(YCrCb_path + '/' + file.name, ycrcb)
    cv2.imwrite(LAB_path + '/' + file.name, lab)
    
    name = file.name.split('.')[0]
    
    h, s, v = cv2.split(hsv)
    cv2.imwrite(HSV_path + '/' + name + '_h.jpg', h)
    cv2.imwrite(HSV_path + '/' + name + '_s.jpg', s)
    cv2.imwrite(HSV_path + '/' + name + '_v.jpg', v)

    y, cr, cb = cv2.split(ycrcb)
    cv2.imwrite(YCrCb_path + '/' + name + '_y.jpg', y)
    cv2.imwrite(YCrCb_path + '/' + name + '_cr.jpg', cr)
    cv2.imwrite(YCrCb_path + '/' + name + '_cb.jpg', cb)

    l, a, b = cv2.split(ycrcb)
    cv2.imwrite(LAB_path + '/' + name + '_l.jpg', l)
    cv2.imwrite(LAB_path + '/' + name + '_a.jpg', a)
    cv2.imwrite(LAB_path + '/' + name + '_b.jpg', b)
    
    b, g, r = cv2.split(image)
    cv2.imwrite(RGB_path + '/' + name + '_b.jpg', b)
    cv2.imwrite(RGB_path + '/' + name + '_g.jpg', g)
    cv2.imwrite(RGB_path + '/' + name + '_r.jpg', r)


In [5]:
files = [file for file in path.iterdir() if file.is_file()]

In [6]:
for file in files:
    other_colors(file, HSV_path, YCrCb_path, LAB_path, RGB_path)

### Changing the contrast and brightness of the image - OP 1

Ref:
- https://docs.opencv.org/3.4/d3/dc1/tutorial_basic_linear_transform.html
- https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#convertscaleabs

In [7]:
LAB_contraste_lum_path = 'tests/LAB_contraste_luminosidade'
Path(LAB_contraste_lum_path).mkdir(parents=True, exist_ok=True)

RGB_contraste_lum_path = 'tests/RGB_contraste_luminosidade'
Path(RGB_contraste_lum_path).mkdir(parents=True, exist_ok=True)

In [43]:
alpha = 1.5 # Contrast control (1.0-3.0)
beta = -80 # Brightness control (0-100)

for file in files:
    image = cv2.imread(file.__str__())
    name = file.name.split('.')[0]
    
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    lnew = cv2.convertScaleAbs(l, alpha=alpha, beta=beta)
    
    limg = cv2.merge((lnew,a,b))
    cv2.imwrite(LAB_contraste_lum_path + '/' + name + '_l.jpg', lnew)
    cv2.imwrite(LAB_contraste_lum_path + '/' + name + '.jpg', limg)
    
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2.imwrite(RGB_contraste_lum_path + '/' + name + '.jpg', final)
    
    b, g, r = cv2.split(final)
    cv2.imwrite(RGB_contraste_lum_path + '/' + name + '_b.jpg', b)
    cv2.imwrite(RGB_contraste_lum_path + '/' + name + '_g.jpg', g)
    cv2.imwrite(RGB_contraste_lum_path + '/' + name + '_r.jpg', r)

### Changing the contrast of the image - OP 2
Ref https://stackoverflow.com/a/41075028

In [8]:
LAB_contraste_path = 'tests/LAB_contraste'
Path(LAB_contraste_path).mkdir(parents=True, exist_ok=True)

RGB_contraste_path = 'tests/RGB_contraste'
Path(RGB_contraste_path).mkdir(parents=True, exist_ok=True)

In [36]:
for file in files:
    image = cv2.imread(file.__str__())
    name = file.name.split('.')[0]
    
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    cv2.imwrite(LAB_contraste_path + '/' + name + '_l.jpg', cl)
    cv2.imwrite(LAB_contraste_path + '/' + name + '.jpg', limg)
    
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2.imwrite(RGB_contraste_path + '/' + name + '.jpg', final)
    
    b, g, r = cv2.split(final)
    cv2.imwrite(RGB_contraste_path + '/' + name + '_b.jpg', b)
    cv2.imwrite(RGB_contraste_path + '/' + name + '_g.jpg', g)
    cv2.imwrite(RGB_contraste_path + '/' + name + '_r.jpg', r)

### Get LAB images

In [9]:
LAB_thresh_path = 'tests/LAB_thresh'
Path(LAB_thresh_path).mkdir(parents=True, exist_ok=True)

LAB with higer contrast

In [10]:
path_lab = Path(LAB_contraste_lum_path)

In [11]:
files_lab = [file for file in path_lab.iterdir() if file.is_file() and '_' not in file.name]

### With the LAB image to threshold

In [19]:
for file in files_lab:
    image = cv2.imread(file.__str__().replace('.jpg', '_l.jpg'))
    
    result_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(result_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imwrite(LAB_thresh_path + '/' + file.name, thresh)
    
    cnts,_ = cv2.findContours(thresh, 1, 2)
    
    cp = image.copy()
    
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        
        if area > 10 and area < 20000:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cp = cv2.drawContours(cp, [box],0,(0,0,255),2)
            
    cv2.imwrite(LAB_thresh_path + '/cp_' + file.name, cp)

### Get RGB images

In [13]:
RGB_thresh_path = 'tests/RGB_thresh'
Path(RGB_thresh_path).mkdir(parents=True, exist_ok=True)

In [14]:
path_rgb = Path(RGB_contraste_path)

In [15]:
files_rgb = [file for file in path_rgb.iterdir() if file.is_file() and '_' not in file.name]

### With RGB images

In [18]:
for file in files_rgb:
    image = cv2.imread(file.__str__().replace('.jpg', '_g.jpg'))
    
    result_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    _,thresh = cv2.threshold(result_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imwrite(RGB_thresh_path + '/' + file.name, thresh)
    
    cnts,_ = cv2.findContours(thresh, 1, 2)
    
    cp = image.copy()
    
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        
        if area > 10 and area < 20000:
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cp = cv2.drawContours(cp, [box],0,(0,0,255),2)
            
    cv2.imwrite(RGB_thresh_path + '/cp_' + file.name, cp)